# Pytorch Text - Language modeling
Notebook for following along with Pytorch Text interpretation tutorial, starting with nn.transformer and torchtext [Pytorch](https://pytorch.org/tutorials/beginner/transformer_tutorial.html) website tutorial.

### Choices for data

<br>

### Libaries and Modules
Importing the necessary libaries and modules for the notebook.

In [1]:
#Import cell
import captum
import copy
import json
import matplotlib as mpl
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import math
import numpy as np
import os, sys
import pandas as pd
import pickle as pk
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms

from typing import Tuple
from torch import nn, Tensor
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.utils.data import dataset
from torchtext.datasets import WikiText2
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

#device = torch.device('cuda' if torch.cuda.is_available else 'cpu')
device = 'cpu' #Cuda having issues on PC, so manual setting to cpu
print(f"Device: {device}")


print("Imports complete")

Device: cpu
Imports complete


<br>

### Importing and preparing data sets
Importing and preparing the data for the models.

In [2]:
#Gather datasets and prepare them for consumption
train_iter = WikiText2(split='train') #"consumed to make vocab"
tokenizer = get_tokenizer('basic_english')
vocab = build_vocab_from_iterator(map(tokenizer, train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

def data_process(raw_text_iter: dataset.IterableDataset) -> Tensor:
    """Converts raw text into a flat Tensors."""
    data = [torch.tensor(vocab(tokenizer(item)), dtype=torch.long) for item in raw_text_iter]
    return torch.cat(tuple(filter(lambda t:t.numel() > 0, data)))


In [3]:
#Importing data sets
train_iter, val_iter, test_iter = WikiText2()
train_data = data_process(train_iter)
val_data = data_process(val_iter)
test_data = data_process(test_iter)

print("Data sets successfully imported.")

Data sets successfully imported.


In [4]:
#Loader definitions
def batchify(data: Tensor, bsz: int) -> Tensor:
    """Divides the data into bsz seperate sequences, removing extra elements that wouldn't
        cleanly fit.   
    Args: 
        data: Tensor, shape [N]
        bsz: int, batch size
        
    Returns:
        Tensor of shap [N // bsz, bsz]"""
    seq_len = data.size(0) // bsz
    data = data[:seq_len * bsz]
    data = data.view(bsz, seq_len).t().contiguous()
    return data.to(device)

batch_size = 20
eval_batch_size = 10
train_data = batchify(train_data, eval_batch_size)
val_data = batchify(val_data, eval_batch_size)
test_data = batchify(test_data, eval_batch_size)
print(f"Loaders defined, running on device: {device}")

Loaders defined, running on device: cpu


In [5]:
#Setting seed value
torch.manual_seed(1247)

<br>

### Class Definitions
<b>Classes:</b><br>
<ul>
    <li>TransformerModel - Language interpretting model.</li>
    <li>PositionalEncoding - Injects information about the relative or absolute position of tokens in the sequence.</li>
</ul>

In [6]:
#Class definition cell
class TransformerModel(nn.Module):
    def __init__(self, ntoken: int, d_model: int, nhead: int, d_hid: int,
                nlayers: int, dropout: float = 0.5) -> None:
        super().__init__()
        self.model_type = "Transformer"
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        encoder_layers = TransformerEncoderLayer(d_model, nhead, d_hid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Embedding(ntoken, d_model)
        self.d_model = d_model
        self.decoder = nn.Linear(d_model, ntoken)
        self.init_weights()
        return None
    
    def init_weights(self) -> None:
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)
        return None
    
    def forward(self, src: Tensor, src_mask: Tensor) -> Tensor:
        """Args: 
            src: Tensor, shape [seq_len, batch_size]
            src_mask: Tensor, shape [seq_len, seq_len]
        Returns:
            output Tensor, shape [seq_len, btch_size, ntoken]"""
        src = self.encoder(src) * math.sqrt(self.d_model)
        src = self.pos_encoder(src)
        print(src[0])
        output = self.transformer_encoder(src, src_mask) #Returning Nans
        print(output[0])
        output = self.decoder(output)
        return output
        
        
class PositionalEncoding(nn.Module):
    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000) -> None:
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)
        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2)*(-math.log(10000.0)/d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position*div_term)
        pe[:, 0, 1::2] = torch.cos(position*div_term)
        self.register_buffer('pe', pe)
        return None
    
    def forward(self, x: Tensor) -> Tensor:
        """Args: 
            x: Tensors, shape[seq_len, batch_size, embedding_dim]"""
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)
        
print("Classes defined.")

Classes defined.


<br>

### Calculation functions
<b>Functions:</b><br>
<ul>
    <li>get_batch - generates a pair of input-target sequences for the transformer model. It subdivides the source data into chunks of length bptt. For language modelling, the model needs the following words as Target.</li>
    <li>generate_square_subsequent_mask - generates an upper triangular matrix of -inf with zeros on the diagonal.
</ul>

In [7]:
#Calculation functions cell
bptt = 35

def get_batch(source: Tensor, i: int) -> Tuple[Tensor, Tensor]:
    """Args:
        source: Tensor, shape [full_seq_len, batch_size]
        i: int
    Returns:
        tuple (data, target), where data has shape [seq)len, batch)size]
            and target has shape [seq_len * batch_size]"""
    seq_len = min(bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].reshape(-1)
    return data, target

def generate_square_subsequent_mask(sz: int) -> Tensor:
        """Generates an upper-triangular matrix of -inf, with zeros on diag."""
        return torch.triu(torch.ones(sz, sz) * float('-inf'), diagonal=1)

print("Calculation functions defined.")

Calculation functions defined.


<br>

### Plotting functions
<b>Functions:</b>
<ul>
    <li></li>
</ul>

In [8]:
#Plotting functions Cell

print("Plotting functions defined.")

Plotting functions defined.


<br>

### Main code
#### Instantiating the model

In [9]:
ntokens = len(vocab)
emsize = 200
d_hid = 200
nlayers = 2
nhead = 2
dropout = 0.2
model = TransformerModel(ntokens, emsize, nhead, d_hid,
    nlayers, dropout).to(device)

#### Running the model
Here CrossEntropyLoss with a stochastic gradient descent optimizer will be used, with an initial learning rate of 5.0.

In [10]:
criterion = nn.CrossEntropyLoss()
lr = 5.0
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

In [11]:
def train(model: nn.Module) -> None:
    model.train()
    total_loss = 0.
    log_interval = 200
    start_time = time.time()
    src_mask = generate_square_subsequent_mask(bptt).to(device)
    
    num_batches = len(train_data)//bptt
    for batch, i in enumerate(range(0, train_data.size(0) - 1, bptt)):
        data, targets = get_batch(train_data, i)
        seq_len = data.size(0)
        if seq_len != bptt: #only on last batch
            src = src_mask[:seq_len, :seq_len]
        output = model(data, src_mask)
        loss = criterion(output.view(-1, ntokens), targets)
        
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()
        
        total_loss += loss.item()
        if batch%log_interval == 0 and batch > 0:
            lr = scheduler.get_last_lr()[0]
            ms_per_batch = (time.time() - start_time)*1000/log_interval
            cur_loss = total_loss/log_interval
            ppl = math.exp(cur_loss)
            print(f'| epoch {epoch:3d} | {batch:5d}/{num_batches:5d} batches | ' 
                  f'lr {lr:02.2f} | ms/batch {ms_per_batch:5.2f} | ' 
                  f'loss {cur_loss:5.2f} | ppl {ppl:8.2f}')
            total_loss = 0
            start_time = time.time()
    return None
    
def evaluate(model: nn.Module, eval_data: Tensor) -> float:
    model.eval()
    total_loss = 0.
    src_mask = generate_square_subsequent_mask(bptt).to(device)
    with torch.no_grad():
        for i in range(0, eval_data.size(0) - 1, bptt):
            data, targets = get_batch(eval_data, i)
            seq_len = data.size(0)
            if seq_len != bptt:
                src_mask = src_mask[:seq_len, :seq_len]
            output = model(data, src_mask)
            output_flat = output.view(-1, ntokens)
            total_loss += seq_len * criterion(output_flat, targets).item()
    print(output)
    return total_loss / (len(eval_data) - 1)

In [12]:
src_mask = generate_square_subsequent_mask(bptt).to(device)
with torch.no_grad():
    data, targets = get_batch(val_data, 0)
    seq_len = data.size(0)
    output = model(data, src_mask)

print(len(output), output[0][0][0])

tensor([[-0.8809,  0.0000,  0.7953,  ...,  0.0911, -1.3320,  2.4423],
        [-0.2253,  0.8223,  0.0000,  ...,  2.4156,  0.0000, -0.1998],
        [ 1.6933,  1.8127, -1.0218,  ...,  0.0000,  0.7724,  0.0000],
        ...,
        [-0.3732,  1.3096, -1.0327,  ...,  0.8822,  1.5489,  2.9052],
        [-0.0000,  0.6479,  0.0000,  ...,  2.2962,  1.1345,  2.6929],
        [ 0.9261, -0.3708, -0.9554,  ...,  0.0000, -1.7135,  0.4621]])
tensor([[-1.1358, -0.4752,  0.2004,  ..., -0.7064, -0.9100,  1.0481],
        [-0.5375,  0.9340, -0.1551,  ...,  1.5628, -1.5987, -1.6020],
        [ 0.8380,  1.3366, -0.6506,  ...,  0.6641, -0.2372, -0.1736],
        ...,
        [ 0.2253,  0.6116,  0.6951,  ...,  0.6623,  0.7245,  0.9509],
        [ 0.3911, -0.4976,  0.1107,  ...,  1.4119,  1.0673,  0.6170],
        [-0.5840, -0.0156, -0.6219,  ..., -2.5978, -2.2353,  0.7475]])
35 tensor(-0.1109)


In [13]:
evaluate(model, val_data[0:21])

tensor([[-0.7047,  0.5901,  0.6362,  ...,  0.0729, -1.0656,  1.9538],
        [-0.1803,  0.6579,  0.3755,  ...,  1.9325,  1.3520, -0.1598],
        [ 1.3546,  1.4502, -0.8175,  ...,  1.4399,  0.6179,  1.1222],
        ...,
        [-0.2986,  1.0477, -0.8262,  ...,  0.7058,  1.2391,  2.3242],
        [-0.7235,  0.5183,  1.0280,  ...,  1.8370,  0.9076,  2.1543],
        [ 0.7409, -0.2966, -0.7643,  ...,  2.0375, -1.3708,  0.3697]])
tensor([[-0.7542,  0.4289, -0.2502,  ..., -0.9617, -1.0325,  1.1304],
        [-0.9748,  0.9484,  0.2931,  ...,  0.9636, -0.4895, -0.8559],
        [ 0.6179,  1.5404, -0.8885,  ...,  0.8260, -0.0944,  0.7981],
        ...,
        [-0.2020,  0.3399,  0.2342,  ...,  0.6860,  1.0774,  1.0988],
        [ 0.0190, -0.0472,  0.3049,  ...,  0.2678,  0.6183,  0.9610],
        [-0.8187,  0.2489, -0.8880,  ..., -0.3667, -1.6860, -0.1413]])
tensor([[[ 0.2076,  2.2348,  0.3500,  ..., -0.6431, -0.0151,  0.7847],
         [-0.5193, -0.8801, -0.4987,  ...,  0.5555,  0.2051, 

10.399749755859375

In [14]:
src = val_data[0:35]
src = model.encoder(src) * math.sqrt(model.d_model)
src = model.pos_encoder(src)
output = model.transformer_encoder(src, src_mask) #Returning Nans
print(src_mask)

tensor([[0., -inf, -inf,  ..., -inf, -inf, -inf],
        [0., 0., -inf,  ..., -inf, -inf, -inf],
        [0., 0., 0.,  ..., -inf, -inf, -inf],
        ...,
        [0., 0., 0.,  ..., 0., -inf, -inf],
        [0., 0., 0.,  ..., 0., 0., -inf],
        [0., 0., 0.,  ..., 0., 0., 0.]])


In [15]:
encoder_layersAlt = TransformerEncoderLayer(emsize, nhead, d_hid, dropout)
transformer_encoderAlt = TransformerEncoder(encoder_layersAlt, nlayers)

In [19]:
src = val_data[0:35]
src = model.encoder(src) * math.sqrt(model.d_model)
src = model.pos_encoder(src)
output = transformer_encoderAlt(src, src_mask) #Returning Nans
print(output)

tensor([[[-2.1260e-01, -2.3389e-01, -1.2937e+00,  ...,  1.8078e+00,
          -1.2606e+00,  1.1940e+00],
         [ 5.6825e-01, -1.3907e+00,  6.9915e-02,  ...,  1.9964e-01,
          -1.3097e+00,  1.3657e-02],
         [ 1.6450e+00,  1.0748e+00, -6.7399e-01,  ...,  1.0723e+00,
          -1.3596e-02,  1.3045e+00],
         ...,
         [-6.4004e-01, -5.7434e-01, -1.2355e+00,  ...,  5.2296e-01,
          -2.1204e-01,  1.1592e+00],
         [-1.6051e-01, -6.7150e-01, -9.8113e-01,  ...,  1.6995e+00,
          -1.5400e+00,  7.9248e-01],
         [ 1.2922e+00, -1.6928e+00, -6.4805e-01,  ...,  1.8692e+00,
          -1.9047e+00,  4.5431e-01]],

        [[ 3.6300e-01, -9.1157e-01,  7.1820e-01,  ..., -1.3750e+00,
          -1.6724e+00,  3.8794e-01],
         [ 1.1176e+00, -6.0291e-01, -3.4432e-01,  ...,  7.4948e-01,
          -2.1611e+00,  7.2425e-01],
         [ 1.1709e+00, -2.1951e-02, -3.5643e-01,  ..., -6.0576e-01,
           9.9619e-02,  1.3806e+00],
         ...,
         [ 7.1644e-02, -1

In [17]:
print(model.transformer_encoder)

TransformerEncoder(
  (layers): ModuleList(
    (0): TransformerEncoderLayer(
      (self_attn): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=200, out_features=200, bias=True)
      )
      (linear1): Linear(in_features=200, out_features=200, bias=True)
      (dropout): Dropout(p=0.2, inplace=False)
      (linear2): Linear(in_features=200, out_features=200, bias=True)
      (norm1): LayerNorm((200,), eps=1e-05, elementwise_affine=True)
      (norm2): LayerNorm((200,), eps=1e-05, elementwise_affine=True)
      (dropout1): Dropout(p=0.2, inplace=False)
      (dropout2): Dropout(p=0.2, inplace=False)
    )
    (1): TransformerEncoderLayer(
      (self_attn): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=200, out_features=200, bias=True)
      )
      (linear1): Linear(in_features=200, out_features=200, bias=True)
      (dropout): Dropout(p=0.2, inplace=False)
      (linear2): Linear(in_features=200, out_features=

In [18]:
x =
#Forced notepad breakpoint

SyntaxError: invalid syntax (Temp/ipykernel_9692/1628758292.py, line 1)

In [20]:
best_val_loss = float('inf')
epochs = 3
best_model = None

for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train(model)
    val_loss = evaluate(model, val_data)
    val_ppl = math.exp(val_loss)
    elapsed = time.time() - epoch_start_time
    print('-'*90)
    print(f'| end of epoch {epoch:3d} | time: {elapsed:5.2f} | '
         f'valid loss {val_loss:5.2f} | valid ppl {val_ppl:8.2f}')
    print('-'*90)
    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = copy.deepcopy(model)
        
    scheduler.step()

tensor([[-0.0000,  0.7376,  0.7953,  ...,  0.0911, -1.3320,  2.4423],
        [-0.0000,  1.0793,  1.3633,  ...,  2.2650,  0.0000,  0.0000],
        [ 0.1468,  1.2471, -0.0000,  ...,  1.0128, -0.7385,  0.2797],
        ...,
        [-0.0000,  0.5210, -0.8532,  ...,  2.2163,  1.4608,  0.5281],
        [ 0.0000,  0.2725, -0.0000,  ...,  1.2965, -0.8597,  1.4617],
        [-0.0000,  1.1809,  0.6958,  ...,  0.9786,  1.5738,  1.9434]],
       grad_fn=<SelectBackward0>)
tensor([[-0.1086, -0.0537, -0.1268,  ..., -1.4850, -0.1336,  0.8568],
        [ 0.2767,  0.6854,  0.7687,  ..., -0.0910, -0.5823, -0.0697],
        [-0.5921,  0.3840, -0.0394,  ...,  0.4418,  0.0520, -0.4178],
        ...,
        [ 0.7640, -0.3803, -1.2771,  ...,  2.1789,  0.8179, -0.1986],
        [-1.0085,  0.9886,  0.5663,  ...,  0.2272, -0.8653, -0.2187],
        [-1.2373,  0.9350,  1.1054,  ...,  0.3397,  0.3877,  0.4484]],
       grad_fn=<SelectBackward0>)
tensor([[ 0.0134, -0.4428, -0.1239,  ..., -0.0501, -0.0000,  0.0

tensor([[-0.5701,  1.6446,  0.0000,  ...,  1.5673,  0.3507,  0.1813],
        [-0.3412,  0.7004, -0.1465,  ...,  2.4738, -0.3818,  2.3439],
        [ 0.8816,  0.0000,  0.1443,  ...,  1.9467, -1.1228,  0.5689],
        ...,
        [ 0.6125,  1.3234,  0.8928,  ..., -0.0434, -0.1617,  2.8628],
        [ 0.0000,  0.0000, -0.6101,  ...,  0.0181,  1.2879,  0.0000],
        [-0.6648,  0.4318,  1.7498,  ...,  1.7248, -0.7686,  0.0781]],
       grad_fn=<SelectBackward0>)
tensor([[-0.3381,  0.1027, -0.0340,  ..., -0.6468, -0.5176, -0.1203],
        [-0.2225,  0.1356, -0.4041,  ..., -0.1942, -1.0522, -0.1022],
        [-0.1244,  0.0644, -0.2062,  ..., -0.6168, -0.9925,  0.0689],
        ...,
        [-0.2544,  0.0701, -0.1135,  ..., -0.4736, -0.7773,  0.0681],
        [-0.4344, -0.1777,  0.0545,  ..., -0.8297, -0.8706,  0.0202],
        [-0.3179,  0.1095,  0.0590,  ..., -0.2981, -0.2178, -0.3367]],
       grad_fn=<SelectBackward0>)
tensor([[-0.5427,  1.6437,  0.3704,  ...,  1.5998,  0.3678,  0.0

       grad_fn=<SelectBackward0>)
tensor([[-0.3249,  1.6296,  0.2812,  ...,  1.5467,  0.4018,  0.0208],
        [-0.0000,  0.7484,  0.0000,  ...,  0.0033, -1.6975,  2.5545],
        [ 1.6763,  2.3526,  0.0000,  ...,  2.2442, -1.4460,  2.8562],
        ...,
        [-0.4916,  2.1750, -1.4194,  ...,  0.6548, -1.1139,  0.5795],
        [ 0.4722, -0.4614,  0.0000,  ...,  1.6439,  0.1920,  0.3959],
        [ 1.0788,  0.9742, -0.8910,  ...,  2.6572, -0.3721,  1.4582]],
       grad_fn=<SelectBackward0>)
tensor([[-0.7030,  0.3770, -0.1114,  ..., -0.1295, -0.1192, -0.1750],
        [-0.2030,  0.0619, -0.1054,  ..., -0.3133, -0.3986,  0.2853],
        [-0.4800,  0.7302, -0.1311,  ...,  0.1112, -0.6252,  0.1894],
        ...,
        [-0.6839,  0.6960, -0.0453,  ..., -0.1095, -0.3468,  0.0064],
        [-0.6089,  0.3948, -0.1688,  ..., -0.0167, -0.2920, -0.1504],
        [-0.6106,  0.0707, -0.2063,  ...,  0.1475, -0.1532,  0.1388]],
       grad_fn=<SelectBackward0>)
tensor([[ 1.7045, -0.2053, -1.

       grad_fn=<SelectBackward0>)
tensor([[-0.3974,  0.7963, -0.0000,  ...,  2.3565, -0.4858,  1.8085],
        [ 1.1128,  0.9652, -0.8401,  ...,  2.7375, -0.3359,  1.4107],
        [-0.3974,  0.7963, -0.2623,  ...,  2.3565, -0.4858,  0.0000],
        ...,
        [-1.5179, -0.4892, -0.7105,  ...,  0.2919,  0.2687,  0.9042],
        [ 0.2788,  1.5979,  0.5210,  ...,  2.1239, -1.2613,  2.7455],
        [ 0.0101,  0.0000, -1.6803,  ...,  2.7241,  0.3358,  2.2351]],
       grad_fn=<SelectBackward0>)
tensor([[-0.9500, -0.0713,  0.0029,  ...,  0.1036, -0.6878,  0.3796],
        [-1.0329, -0.2311, -0.6597,  ..., -0.2621, -0.7388,  0.1978],
        [-0.6516, -0.0188, -0.4672,  ..., -0.0050, -0.6570,  0.1720],
        ...,
        [-1.0359,  0.0118, -0.3904,  ..., -0.4625, -0.8481, -0.0174],
        [-0.0660,  0.1275, -0.3937,  ..., -0.4150, -0.6562,  0.1738],
        [-0.9545, -0.0094, -0.4301,  ..., -0.2313, -0.0462,  0.1557]],
       grad_fn=<SelectBackward0>)
tensor([[-0.0479,  1.6651, -0.

       grad_fn=<SelectBackward0>)
tensor([[-0.5606, -0.1709, -0.5362,  ..., -0.2806, -0.4275,  0.2975],
        [-0.8414, -0.1693, -0.0898,  ...,  0.1485,  0.0240,  0.3893],
        [-0.0186, -0.0127, -0.2339,  ..., -0.0834, -0.0815,  0.6689],
        ...,
        [-0.1056, -0.1644, -0.7049,  ..., -0.1177, -0.5620,  0.7034],
        [-0.6209,  0.1194, -0.4393,  ...,  0.0084, -0.4461,  0.0466],
        [-0.6710, -0.0564, -0.3710,  ..., -0.2276, -0.3038,  0.3963]],
       grad_fn=<SelectBackward0>)
tensor([[ 0.7847,  0.7820, -0.8498,  ...,  2.8869, -0.5131,  0.6249],
        [-0.4693,  0.9284,  0.4542,  ...,  0.0000,  1.6141, -0.5494],
        [ 1.7604,  0.0000, -0.0000,  ...,  2.2972,  0.0000,  0.7101],
        ...,
        [-0.1327,  2.6065, -1.3278,  ...,  0.6142, -0.9705,  0.0614],
        [ 1.4281,  0.0000, -0.7963,  ...,  0.0000, -0.9407,  2.5482],
        [ 1.1747,  2.8374, -0.3233,  ...,  1.4586, -0.0000,  1.6979]],
       grad_fn=<SelectBackward0>)
tensor([[-0.3569, -0.2049, -0.

       grad_fn=<SelectBackward0>)
tensor([[-0.8075, -0.2948, -0.4248,  ..., -0.0702, -0.3401,  0.6224],
        [-0.4782, -0.3814, -0.5946,  ..., -0.3074, -0.3006,  0.4344],
        [-0.4470,  0.2075, -0.7323,  ..., -0.0158, -0.3094,  0.3129],
        ...,
        [-0.5981,  0.3384, -0.4217,  ..., -0.3580, -0.1467,  0.3741],
        [-0.5735,  0.0547, -0.4516,  ..., -0.3018, -0.2439,  0.5268],
        [-0.4917, -0.2168, -0.4356,  ..., -0.1713, -0.3383,  0.6420]],
       grad_fn=<SelectBackward0>)
tensor([[ 0.0000,  0.3097, -1.3776,  ...,  0.0000,  0.3327,  0.5239],
        [ 0.5768, -0.3998,  0.0356,  ..., -0.6002, -0.0000,  0.2713],
        [-1.5201,  1.7955, -1.3068,  ...,  1.3496, -0.0189,  2.9341],
        ...,
        [-1.0437, -0.2709, -0.4069,  ...,  0.9696, -0.5069,  2.5903],
        [ 1.0562,  0.9820,  0.8302,  ...,  2.6928, -1.0534, -0.2202],
        [-1.2873,  1.4754, -0.0000,  ..., -0.4548,  0.6815,  0.9373]],
       grad_fn=<SelectBackward0>)
tensor([[-0.7459, -0.0448, -0.

tensor([[-0.6131,  0.2537, -0.4244,  ..., -0.1981, -0.1966,  0.2535],
        [-0.6542,  0.2483, -0.5824,  ..., -0.0528, -0.2210,  0.3320],
        [-0.6791,  0.2320,  0.0371,  ..., -0.1141, -0.4394, -0.0535],
        ...,
        [-0.5905,  0.3813, -0.5121,  ..., -0.2183, -0.2301, -0.0773],
        [-0.8351,  0.2524, -0.0879,  ..., -0.2198, -0.5709,  0.4171],
        [-0.5352,  0.0778, -0.5617,  ..., -0.0487,  0.0770, -0.1119]],
       grad_fn=<SelectBackward0>)
tensor([[-1.0648,  1.7342, -0.5454,  ...,  1.1699,  1.1234,  0.1317],
        [ 0.0550,  0.4451,  0.0000,  ...,  1.7211,  1.2036,  0.0000],
        [-0.7282,  1.3639,  1.1112,  ...,  2.7090,  0.7565,  0.0000],
        ...,
        [ 1.3867,  0.0000, -0.0555,  ...,  1.0206, -1.2010,  1.5804],
        [ 0.1910,  1.0452,  0.3191,  ...,  0.0000,  0.0000,  0.8017],
        [-0.0000,  0.2526, -0.4141,  ...,  2.4861,  0.0000,  0.0000]],
       grad_fn=<SelectBackward0>)
tensor([[-0.2636,  0.4415, -0.4339,  ..., -0.2775, -0.3001,  0.3

       grad_fn=<SelectBackward0>)
tensor([[-0.4712, -0.0781, -0.0179,  ..., -0.1462, -0.0563,  0.3902],
        [-0.4506, -0.1274, -0.4169,  ..., -0.1208, -0.0453, -0.0264],
        [-0.2860, -0.1024,  0.2893,  ..., -0.1515, -0.3272,  0.4286],
        ...,
        [-0.3596, -0.2190,  0.0338,  ..., -0.1155, -0.1868, -0.0765],
        [ 0.0550,  0.1199, -0.4492,  ..., -0.1584, -0.4720, -0.0248],
        [ 0.2187,  0.0819,  0.1670,  ...,  0.1565, -0.2062,  0.5921]],
       grad_fn=<SelectBackward0>)
tensor([[-0.8940,  2.4214,  1.7337,  ...,  0.0554, -0.0000,  1.4428],
        [ 1.0269,  0.0000, -0.7386,  ...,  2.7677, -0.1209,  1.3141],
        [-0.0000,  2.2834, -0.3383,  ..., -0.1641,  0.4620,  1.5949],
        ...,
        [ 0.7719,  0.4140,  1.4151,  ...,  0.0000,  0.9892, -0.0856],
        [ 0.0000,  0.0000,  0.2165,  ...,  2.2070,  1.0072,  0.8032],
        [-0.0000,  1.6266, -0.5798,  ...,  0.0000,  1.2693,  0.0073]],
       grad_fn=<SelectBackward0>)
tensor([[-0.6789, -0.1285,  0.

       grad_fn=<SelectBackward0>)
tensor([[-0.4966,  0.1232,  0.3384,  ...,  1.9700, -1.5854,  2.9314],
        [ 1.0939,  2.8384, -1.4971,  ..., -0.0000,  1.7866,  3.4860],
        [ 1.1108,  0.0000,  0.1483,  ..., -1.0978, -0.7937, -0.7506],
        ...,
        [-0.8908,  1.8810, -0.5764,  ...,  0.9389,  0.0000, -0.5049],
        [ 0.3569,  1.6687, -0.4146,  ...,  1.3643, -0.0000,  0.9484],
        [ 1.3064,  0.2804, -0.4243,  ...,  1.1262, -1.3343,  1.4172]],
       grad_fn=<SelectBackward0>)
tensor([[-0.7494, -0.2790, -0.1216,  ...,  0.3072, -0.8617,  0.6881],
        [-0.3817,  0.4374, -0.6313,  ...,  0.1892, -0.0492,  0.8540],
        [ 0.2145,  0.3698, -0.6415,  ..., -1.0970, -0.4046,  0.0391],
        ...,
        [-0.4458,  0.3632, -0.1943,  ...,  0.3010,  0.0088, -0.0099],
        [-0.0951,  1.6238, -0.1296,  ...,  0.4509, -0.6925, -0.9546],
        [-0.4492, -0.2107, -0.2442,  ..., -0.0151, -0.7136,  0.2578]],
       grad_fn=<SelectBackward0>)
tensor([[ 0.4604,  4.7819, -0.

KeyboardInterrupt: 

<br>